In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from torch import nn

import pandas as pd

import numpy as np

import torch

smooth = SmoothingFunction()

max_length = 128

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-zh", model_max_length = max_length)

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-zh")

'''
class BertTransModel(nn.Module):
    def __init__(self):

        super(BertTransModel,self).__init__()

        self.bert = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-zh")
        
    def forward(self, ids, mask, tgt, mask1):

        out, _ = self.bert(input_ids = ids, attention_mask = mask, decoder_ids = tgt)

        return out
'''

#mymodel = BertTransModel()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#mymodel.to(device)

model.to(device)


ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [ ]:
train_df = pd.read_csv('C:/Users/z1055/Desktop/ClassDocument/NLP/Experiment6/train.csv',sep = ',',names = ['text','label'],skiprows = 1)

print(train_df.shape)

count = train_df.shape[0]

train_df.head()

sentences = list(train_df['text'])
    
targets = list(train_df['label'])

In [3]:
def freeze_layers(model,freeze_list):

    for name, child in model.named_children():

        if name not in ['model']:

            continue

        for name1, child1 in child.named_children():

            if name1 not in ['decoder']:

                continue

            for name2, child2 in child1.named_children():

                if name2 not in ['layers']:

                    continue

                for name3, child3 in child2.named_children():

                    if(int(name3) not in freeze_list):
                        
                        continue

                    for para in child3.parameters():

                        para.requires_grad = False

freeze_layers(model,[5])

In [4]:

from torch import optim

from transformers import AdamW
#optimizer = optim.Adam(mymodel.parameters(), lr = 0.0001)
#optimizer = optim.Adam(model.parameters(), lr = 0.0001)
optimizer = AdamW(model.parameters(), lr = 0.00001)

epochs = 3

batch_size = 32

model.train()

for epoch in range(epochs):

    train_loss = 0.0

    rangeNum = int(count / batch_size + 1)

    for val in range(rangeNum):

        if val * batch_size >= count:
            break

        if val == rangeNum - 1:

            src_batch = tokenizer.prepare_seq2seq_batch(src_texts = sentences[val * batch_size:],padding=True)
            
            with tokenizer.as_target_tokenizer():

                tgt_batch = tokenizer.prepare_seq2seq_batch(src_texts = targets[val * batch_size:],padding=True)

        else:

            src_batch = tokenizer.prepare_seq2seq_batch(src_texts = sentences[val * batch_size:batch_size * (val + 1)], padding=True)

            with tokenizer.as_target_tokenizer():

                tgt_batch = tokenizer.prepare_seq2seq_batch(src_texts = targets[val * batch_size:batch_size * (val + 1)], padding=True)

        src_batch["input_ids"] = torch.tensor(src_batch["input_ids"][:][:max_length]).to(device).detach()

        tgt_batch["input_ids"] = torch.tensor(tgt_batch["input_ids"][:][:max_length]).to(device).detach()

        optimizer.zero_grad()

        loss = model(input_ids = src_batch["input_ids"],  labels = tgt_batch["input_ids"], ).loss

        loss.backward()

        optimizer.step()

        if (val + 1) % 100 == 0:

            print('[Epoch = %d Iternum = %d Loss = %f]'%(epoch,val + 1, loss))


C:\ProgramData\Miniconda3\lib\site-packages\transformers\tokenization_utils_base.py:3255: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(
[Epoch = 0 Iternum = 100 Loss = 1.414109]
[Epoch = 0 Iternum = 200 Loss = 1.198318]
[Epoch = 0 Iternum = 300 Loss = 1.640920]
[Epoch = 0 Iternum = 400 Loss = 1.605938]
[Epoch = 0 Iternum = 500 Loss = 2.023006]
[Epoch = 0 Iternum = 600 Loss = 1.233098]
[Epoch = 0 Iternum = 700 Loss = 1.479905]
[Epoch = 0 Iternum = 800 Loss = 1.745520]
[Epoch = 0 Iternum = 900 Loss = 1.472933]
[Epoch = 0 Iternum = 1000 Loss = 1.366561]
[Epoch = 0 Iternum = 1100 Loss = 1.526798]
[Epoch = 0 Iternum = 1200 Loss = 1.758339]
[Epoch = 0 Iternum = 1300 Loss = 1.574931]
[Epoch = 0 Iternum = 1

In [16]:
def calculate_bleu(pred_trgs, trgs):

    res = 0

    count = 0

    for candidate,reference in zip(pred_trgs,trgs):

        count = count + 1

        #score = sentence_bleu([reference], candidate,smoothing_function = smooth.method1,weights=(0.25, 0.25, 0.25, 0.25))

        score = sentence_bleu([reference], candidate, weights=(0.25, 0.25, 0.25, 0.25))


        res = res + score
        
    return res

In [17]:
def evaluate(file_dir,batch_size,tokenizer,model,num_beams):

    model.eval()

    pred = ''

    src_list = []

    tgt_list = []

    count = 0

    with open(file_dir,'r',encoding = 'utf-8') as sentence_file:

        for sentence in sentence_file.read().split('\n'):

            count = count + 1

            if(count % 2 == 0):

                tgt_list.append(sentence)

            else:

                src_list.append(sentence)

    print('Processing Complete')

    rangeNum = int(count / batch_size / 2 + 1)

    score = 0

    for val in range(rangeNum):

        if val * batch_size >= count / 2:

            break

        if val == rangeNum - 1:

            batch = tokenizer.prepare_seq2seq_batch(src_texts=src_list[val * batch_size:])

        else:

            batch = tokenizer.prepare_seq2seq_batch(src_texts=src_list[val * batch_size:batch_size * (val + 1)])

        batch["input_ids"] = torch.tensor(batch["input_ids"][:][:max_length]).to(device).detach()

        batch["attention_mask"] = torch.tensor(batch["attention_mask"][:][:max_length]).to(device).detach()

        translation = model.generate(**batch, num_beams = num_beams)

        result = tokenizer.batch_decode(translation, skip_special_tokens=True, num_beams = num_beams)

        if val == rangeNum - 1:

            score = score + calculate_bleu(result,tgt_list[val * batch_size:])

        else:

            score = score + calculate_bleu(result,tgt_list[val * batch_size:batch_size * (val + 1)])

        if val % 5 == 0:

            print('[Iternum = %d BLEU SCORE = %f]'%(val,score * 100.0 / ((val + 1) * batch_size)))

    return score * 100.0 / count * 2

In [18]:
save_directory = "C:/Users/z1055/Desktop/ClassDocument/NLP/Experiment6/models/best_model"

#tokenizer = AutoTokenizer.from_pretrained(save_directory)

#model = AutoModelForSeq2SeqLM.from_pretrained(save_directory).to(device)

res = evaluate(file_dir = 'C:/Users/z1055/Desktop/ClassDocument/NLP/Experiment6/valid.txt',batch_size = 8,tokenizer = tokenizer,model = model,num_beams = 10)

print('Res = ',res)

Processing Complete
[Iternum = 0 BLEU SCORE = 16.173985]
[Iternum = 5 BLEU SCORE = 16.348367]
[Iternum = 10 BLEU SCORE = 16.271617]
[Iternum = 15 BLEU SCORE = 17.510386]
[Iternum = 20 BLEU SCORE = 19.046815]
[Iternum = 25 BLEU SCORE = 19.529813]
[Iternum = 30 BLEU SCORE = 19.122892]
[Iternum = 35 BLEU SCORE = 18.741071]
[Iternum = 40 BLEU SCORE = 19.237890]
[Iternum = 45 BLEU SCORE = 19.680164]
[Iternum = 50 BLEU SCORE = 19.947722]
[Iternum = 55 BLEU SCORE = 19.630131]
[Iternum = 60 BLEU SCORE = 19.514640]
[Iternum = 65 BLEU SCORE = 19.130647]
[Iternum = 70 BLEU SCORE = 19.307333]
[Iternum = 75 BLEU SCORE = 19.219137]
[Iternum = 80 BLEU SCORE = 18.828400]
[Iternum = 85 BLEU SCORE = 18.697297]
[Iternum = 90 BLEU SCORE = 18.699316]
[Iternum = 95 BLEU SCORE = 18.926841]
[Iternum = 100 BLEU SCORE = 18.840455]
[Iternum = 105 BLEU SCORE = 18.897204]
[Iternum = 110 BLEU SCORE = 18.928305]
[Iternum = 115 BLEU SCORE = 19.013273]
[Iternum = 120 BLEU SCORE = 18.997963]
[Iternum = 125 BLEU SCORE =

In [8]:
#save_directory = "C:/Users/z1055/Desktop/ClassDocument/NLP/Experiment6/models/best_model"

#tokenizer.save_pretrained(save_directory)

#model.save_pretrained(save_directory)

In [9]:
def get_pred(file_dir,pred_file_dir,batch_size,tokenizer,model,num_beams):

    model.eval()

    pred = ''

    src_list = []

    count = 0

    with open(file_dir,'r',encoding = 'utf-8') as sentence_file:

        for sentence in sentence_file.read().split('\n'):

                src_list.append(sentence)

                count = count + 1

    print('Processing Complete')

    rangeNum = int(count / batch_size + 1)

    score = 0

    with open(pred_file_dir,'w') as sentence_file:

        for val in range(rangeNum):

            if val * batch_size >= count:

                break

            if val == rangeNum - 1:

                batch = tokenizer.prepare_seq2seq_batch(src_texts=src_list[val * batch_size:])

            else:

                batch = tokenizer.prepare_seq2seq_batch(src_texts=src_list[val * batch_size:batch_size * (val + 1)])

            batch["input_ids"] = torch.tensor(batch["input_ids"][:][:max_length]).to(device).detach()

            batch["attention_mask"] = torch.tensor(batch["attention_mask"][:][:max_length]).to(device).detach()

            translation = model.generate(**batch, num_beams = num_beams)

            result = tokenizer.batch_decode(translation, skip_special_tokens=True, num_beams = num_beams)

            for sentence_result in result:

                sentence_file.write(sentence_result + '\n')

            print(result)

In [10]:
save_directory = "C:/Users/z1055/Desktop/ClassDocument/NLP/Experiment6/models/best_model"

tokenizer = AutoTokenizer.from_pretrained(save_directory)

model = AutoModelForSeq2SeqLM.from_pretrained(save_directory).to(device)

test_file_dir = 'C:/Users/z1055/Desktop/ClassDocument/NLP/Experiment6/eval/test1.txt'

pred_file_dir = 'C:/Users/z1055/Desktop/ClassDocument/NLP/Experiment6/eval/pred1.txt'

get_pred(file_dir = test_file_dir, pred_file_dir = pred_file_dir, batch_size = 8, tokenizer = tokenizer, model = model, num_beams = 10)

Processing Complete
['我们喜欢踢足球。', '我们尽了一切努力去帮助他。', '嘿,你的狗咬了我。', '汤姆现在在一家银行工作。', '我注意到她戴着一顶新帽子。', '你用什么打开的?', '我们的棒球队很强壮。', '你想知道为什么我对汤姆说谎吗?']
['别听她的。', '他娶了一个空姐。', '什么是爱?', '汤姆这次做得比上次好。', '恭喜!', '还剩多少时间?', '我们两个是学生。', '如果你住在大旅馆,你可以使用他们的游泳池。']
['你能忍受他的行为举止吗?', '这个城市的人口大约是10万。']
